In [1]:
# Load the Drive helper and mount
#from google.colab import drive

# This will prompt for authorization.
#drive.mount('/content/drive')# Load the Drive helper and mount

In [2]:
# data_path = "data/04256520"#"/content/drive/My Drive/Kaist/[2022]CS492/Implementation/04256520"

## Library Import

In [12]:
# Library Import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.utils.data as data_utils
import random
import tqdm
import numpy as np
#import sklearn
#import pandas as pd
import os
torch.cuda.empty_cache()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # CHANGED BY OLIVIA
os.environ["CUDA_VISIBLE_DEVICES"]="6" # CHANGED BY OLIVIA
import gc
gc.collect()
torch.cuda.empty_cache()
import time

## Data Loader

In [6]:
DIRECTORY_MODELS = 'data/SdfSamples/ShapeNetV2/04256520' #'data/SdfSamples/ShapeNetV2/04256520'
MODEL_EXTENSION = '.npz'
def get_model_files():
    for directory, _, files in os.walk(DIRECTORY_MODELS):
        for filename in files:
            if filename.endswith(MODEL_EXTENSION):
                yield  os.path.join(filename)
    
def get_model_dir():
    for directory, _, files in os.walk(DIRECTORY_MODELS):
        for filename in files:
            if filename.endswith(MODEL_EXTENSION):
                yield os.path.join(directory, filename)
                
files = list(get_model_files())
dirs = list(get_model_dir())

print("the number of Dataset:", len(files))

the number of Dataset: 3173


In [2]:
num_train = 1000
num_test = 100
# train set!
files_dir_train = dirs[:num_train] # /data.../...npz
files_npz_train = files[:num_train] # ...npz
len(files_npz_train)
files_dir_test = dirs[num_train+1:num_train+num_test+1] # /data.../...npz
files_npz_test = files[num_train+1:num_train+num_test+1] # ...npz

print("the number of Dataset(train):", len(files_npz_train))
print("the number of Dataset(test):", len(files_npz_test))

NameError: name 'dirs' is not defined

In [9]:
mesh_directory = 'mesh/'
latent_directory = 'latent/'
for i, npz in enumerate(files_npz_train):
    if "npz" not in npz:
        continue
    mesh_fname = os.path.join(mesh_directory, npz[:-4] + ".ply") #mesh 
    latent_fname = os.path.join(latent_directory, npz[:-4] + ".pth") #latent
    #result directory

In [10]:
# code from the DeepSDF Git
# https://github.com/facebookresearch/DeepSDF/blob/48c19b8d49ed5293da4edd7da8c3941444bc5cd7/deep_sdf/data.py


def sdf_dispenser(data, subsample=None):
    if subsample is None:
        return data
    pos_tensor = data[0]
    neg_tensor = data[1]

    # split the sample into half
    half = int(subsample / 2)

    pos_size = pos_tensor.shape[0]
    neg_size = neg_tensor.shape[0]

    pos_start_ind = random.randint(0, pos_size - half)
    sample_pos = pos_tensor[pos_start_ind : (pos_start_ind + half)]

    if neg_size <= half:
        random_neg = (torch.rand(half) * neg_tensor.shape[0]).long()
        sample_neg = torch.index_select(neg_tensor, 0, random_neg)
    else:
        neg_start_ind = random.randint(0, neg_size - half)
        sample_neg = neg_tensor[neg_start_ind : (neg_start_ind + half)]

    samples = torch.cat([sample_neg, sample_pos], 0)
    #Random Shuffle!
    randidx = torch.randperm(samples.shape[0])
    samples = torch.index_select(samples, 0, randidx)

    return samples

def remove_nans(tensor):
    tensor_nan = torch.isnan(tensor[:, 3])
    return tensor[~tensor_nan, :]


class SDFSamples(torch.utils.data.Dataset):
    def __init__(
        self,
        npyfiles,
        subsample,
        print_filename=False,
        num_files=1000000,
    ):
        self.subsample = subsample

        self.data_source = data_source
        self.npyfiles = npyfiles
        self.loaded_data = []
        for f in tqdm.tqdm(self.npyfiles, ascii=True):
            filename = os.path.join(f)
            npz = np.load(filename)
            pos_tensor = remove_nans(torch.from_numpy(npz["pos"]))
            neg_tensor = remove_nans(torch.from_numpy(npz["neg"]))

            
            #Random Shuffle
            self.loaded_data.append(
                [
                    pos_tensor[torch.randperm(pos_tensor.shape[0])],
                    neg_tensor[torch.randperm(neg_tensor.shape[0])],
                ])

    def __len__(self):
        return len(self.npyfiles)

    def __getitem__(self, idx):
        filename = self.npyfiles[idx]
        return (
            sdf_dispenser(self.loaded_data[idx], self.subsample),
            idx,
        )
    
data_source = DIRECTORY_MODELS
file_dirs = files_dir_train
num_samp_per_scene = 5000 #00 #
scene_per_batch = 1 #Change!
num_data_loader_threads = 1
npyfiles = files_dir_train # /data.../...npz

sdf_data=SDFSamples(npyfiles,num_samp_per_scene)
# Data Load
sdf_loader = data_utils.DataLoader(
            sdf_data,
            batch_size=scene_per_batch,
            shuffle=True,
            num_workers=0,
            drop_last=True, #Don't use the Last Batch
            )

100%|##########| 1000/1000 [00:13<00:00, 71.81it/s]


In [11]:
# Data Load
sdf_loader = data_utils.DataLoader(
            sdf_data,
            batch_size=scene_per_batch,
            shuffle=True,
            num_workers=0,
            drop_last=True,
            )

## CUDA SETTING

In [5]:
# use cuda GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("used device", device)

used device cuda


In [8]:
#!nvidia-smi
device

device(type='cuda')

## Deep Implicit Templates Model
### LSTM & MLP

In [3]:
# From the DIT github
# https://github.com/ZhengZerong/DeepImplicitTemplates

#weight initialization 
def weight_initial(self):
    for m in self.modules():
        for n, p in m.named_parameters():
            if 'weight_ih' in n:
                nn.init.kaiming_normal_(p.data)
            elif 'weight_hh' in n:
                nn.init.orthogonal_(p.data)
            elif 'bias' in n:
                p.data.fill_(0)
            
#output weight initialization
def weights_out_init(self):
    for m in self.modules():
        for n, p in m.named_parameters():
            if 'weight' in n:
                nn.init.uniform_(p.data, -1e-5, 1e-5)
            elif 'bias' in n:
                nn.init.constant_(p.data, 0)

In [9]:
# From the DIT github
# https://github.com/ZhengZerong/DeepImplicitTemplates

class Warper(nn.Module):
    def __init__(
            self,
            steps,
            latent_size,
            h_size,
            dim,
            linee = False
    ):
        super(Warper, self).__init__()
        #latent vector + xyz coordinates
        out_len = 6
        self.n_feature_channels = latent_size + 3
        self.h_size = h_size
        self.steps = steps
        self.lstm = nn.LSTMCell(input_size=self.n_feature_channels,
                                hidden_size=h_size)
        lstm_layer = self.lstm
        out_len = 6
        
        #LSTM Cell weight initial
        lstm_layer.apply(weight_initial)
        for n, p in lstm_layer.named_parameters():
            if "bias" in n: # continue
                si = p.size(0)
                start, end = si // 4, si // 2
                p.data[start:end].fill_(1.)

        self.out_feature = nn.Linear(h_size, out_len)
        # Out Layer Weight Initializer
        self.out_feature.apply(weights_out_init)

    def forward(self, input):
        xyz = input[:, -3:] #Change
        lat = input[:, :-3] # Fix
        states = [None]
        warped_xyzs = []
        warping_p = []

        # Step for 8 times
        for s in range(self.steps):
            cell_st = self.lstm( torch.cat([lat, xyz], dim=1), states[-1]) #Latest state!
            rgd = cell_st[0].requires_grad
            if rgd:
                cell_st[0].register_hook(lambda x: x.clamp(min=-10, max=10))
            a = self.out_feature(cell_st[0])
            
            int_xyz = torch.addcmul(a[:, 3:], (a[:, :3]+1), xyz)
                                    #SDF + 1*(1+ w_xyz)*xyz
            xyz_t = int_xyz
            if (s+1) % 2 == 0:
                warped_xyzs.append(xyz_t)
            states.append(cell_st)
            warping_p.append(a)

                #중간에 있는 값들!
            xyz = int_xyz
        return xyz, warping_p, warped_xyzs 
        #latest xyz, parameter, intermediate xyzs

In [10]:
# MLP : SDF Decoder

class MLP(nn.Module):
    def __init__(self, layers, wn_layers, weight_norm, dropout_layers, dropout_prob):
        """
            Initialize the MLP that extracts the template SDF
        """
        
        # layers: number of layers + size of each layer
        # [256, 256, 256, 256, 256] 
        # => 5 layers with 256 neurons each
        
        # wn_layers: layer indices in which normalization is used
        # [0, 1, 2, 3, 4]
        
        # weight_norm: bool
        # which normalization to use????? TODO
        
        # dropout_layers: layer indices in which dropout is used
        # [0, 1, 2, 3, 4]
        
        # dropout_prob: probability for dropout
        # 0.05
        
        super(MLP, self).__init__()
        
        
        self.numlayers = len(layers)+1
        #print("numlayers", self.numlayers)
        
        in_dim = 3
        out_dim = 1
        
        self.layers = nn.ModuleList()
        for i in range(self.numlayers):
            #print(i)
            # layer input feature count (for first layer: in_dim)
            in_features = in_dim if (i == 0) else layers[i-1]
            # layer output feature count (for last layer: out_dim)
            out_features = out_dim if (i == (len(layers))) else layers[i]
            
            # fully connected layer
            layer = nn.Linear(in_features, out_features)
            
            modules = [layer]
            if i in wn_layers:
                # weight normalization layer
                if False:  ##########weight_norm: ########## TODO
                    layer = nn.utils.weight_norm(layer) # ????????
                    modules[0] = layer
                else:
                    modules.append(nn.LayerNorm(out_features))
                    modules.append(nn.BatchNorm1d(out_features)) #ADD!
            
            # activation (tanh if last layer, else relu)
            activation = nn.Tanh() if (i == (len(layers))) else nn.ReLU()
            modules.append(activation)
            
            # dropout
            if i in dropout_layers: ### TODO only if training??
                modules.append(nn.Dropout(dropout_prob))
            
            
            # if last layer (and tanh) => another tanh??
            #if (i == (len(layers))): #### TODO and tanh
            #    modules.append(nn.Tanh())
            
            sequential = nn.Sequential(*modules)
            self.layers.append(sequential)
            
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
        return x.squeeze()

In [11]:
# output: warped point (from warper) and predicted SDF (from MLP)

class DIT(nn.Module):
    def __init__(self, mlp_args, lstm_args):
        """
            the entire network consisting of the LSTM and the MLP
        """
        super(DIT, self).__init__()
        self.mlp = MLP(**mlp_args)
        self.lstm = Warper(**lstm_args)
        
        #print("mlp weigths", self.mlp.state_dict())
        #print("lstm weights", self.lstm.state_dict())
      
    def forward(self, x):
        # get warped points
        t_lstm = time.time()
        p_canonical, warping_param, intermediate_xyzs = self.lstm(x)

        if self.training:
            # get sdf value for each intermediate warping step
            sdf_values = []
            for points in intermediate_xyzs:
                sdf = self.mlp(points)
                sdf_values.append(sdf.squeeze())
            return sdf_values, intermediate_xyzs
        else:
            # only final sdf value
            print("no_train")
            sdf_value = self.mlp(p_canonical)
            print("sdf_value",sdf_value)
            return sdf_value, p_canonical


## Loss Function
### Regularization Loss, Reconstruction Loss

In [19]:
# Loss Function code
# DeepSDF: https://github.com/facebookresearch/DeepSDF
# Deep Implicit Templates: https://github.com/ZhengZerong/DeepImplicitTemplates

#----------------------------------------------------------------------
#                         Reconstruction loss
#----------------------------------------------------------------------
def reconstruction_loss(sdf_value_list, ground_truth):
    #print("reconstruction_loss")

    eps_list = [0.025, 0.01, 0.0025, 0]
    lambda_list = [0, 0.1, 0.2, 0.5]
    loss = []
    for s in range(len(eps_list)): # for each warping step 
        # loss from curriculum DeepSDF
        # https://arxiv.org/pdf/2003.08593.pdf
        dist = ground_truth - sdf_value_list[s]
        loss_e = torch.relu(torch.abs(dist) - eps_list[s])
        curriculum_loss = (1 + lambda_list[s] * torch.sign(ground_truth) * torch.sign(dist)) * loss_e
        loss.append(torch.mean(curriculum_loss))  #TODO sum??
        
    return sum(loss) / len(loss) # average loss across all the warping steps

#----------------------------------------------------------------------
#                         Regularization loss
#----------------------------------------------------------------------
def huber_kernel(a, delta):
    # https://en.wikipedia.org/wiki/Huber_loss
    abs = torch.abs(a)
    return torch.where(abs <= delta, (a*a) / 2, delta * (abs - delta/2))

def regularization_loss(original_points, warped_points, latent_codes, epoch):
    pw_weight = 0.005 ### from git (sofa)
    pp_weight = 0.0001 ### from git (sofa)
    epsylon = 0.5 ### From paper
    code_lambda = 1e-4 # DeepSDF
    num_of_samples = len(warped_points)
    start = time.time()
    # --------------------------
    # point wise regularization
    # --------------------------
    
    # distances of original and warped points
    delta_p = warped_points - original_points

    # normalize with L2-norm
    distances_normalized = torch.linalg.norm(delta_p, dim=1, ord=2)

    # apply huber kernel and sum up for all samples
    pw_loss = torch.sum(huber_kernel(distances_normalized, delta = 0.25)) / num_of_samples #TODO remove / num_of_samples ?
   # print("Reg Loss: ", time.time() - start)

    # --------------------------
    # point wise regularization
    # --------------------------
    # from git!
    
    # compare point k to all the other points?
    k = num_of_samples//8 #from DIT implementation on git
    start = time.time()
    # || delta_p_i - delta_p_j ||2
    distances_diff = delta_p[:k].view(-1,1,3) - delta_p[k:].view(1,-1,3)
    norm_delta_p = torch.linalg.norm(distances_diff, dim=2, ord=2)
    # || p_i - p_j ||2
    start = time.time()
    point_diff = original_points[:k].view(-1,1,3) - original_points[k:].view(1,-1,3)
    norm_p = torch.linalg.norm(point_diff, dim=2, ord=2) # + 1e-3 ? to not divide by 0
    
    start = time.time()
    # only positive values
    pp_loss = torch.sum(torch.nn.functional.relu(norm_delta_p/norm_p - epsylon).flatten()) / num_of_samples #TODO remove / num_of_samples ?
    
    # --------------------------
    # latent code magnitude loss
    # --------------------------
    start = time.time()
    
    # from DeepSDF
    code_norm = torch.sum(torch.norm(latent_codes, dim=1)) / num_of_samples
    latent_code_loss = code_lambda* min(1.0, epoch/100) * code_norm.cuda()   ### TODO 1/std^2 sum over k ||ck||2^2
    return pw_weight * pw_loss + pp_weight * pp_loss + latent_code_loss


In [20]:
log_dir = ""
from torch.utils.tensorboard import SummaryWriter 
writer = SummaryWriter(log_dir)

## Training!

In [ ]:
# initialize Network
from easydict.easydict import EasyDict as edict
#from easydict import EasyDict as edict

lstm_args = edict()
lstm_args.steps = 8
lstm_args.latent_size = 256
lstm_args.h_size = 512
lstm_args.dim = [512, 64]

lat_vecs = torch.nn.Embedding(len(sdf_loader), lstm_args.latent_size, max_norm=1)
#torch.nn.init.normal_(lat_vecs.weight.data, 0.0, 1.0/math.sqrt(lat_vecs))

mlp_args = edict()
mlp_args.layers = [256,256,256,256,256]
mlp_args.wn_layers = [0,1,2,3,4]
mlp_args.weight_norm = True
mlp_args.dropout_layers =[0,1,2,3,4]
mlp_args.dropout_prob = 0.05

decoder = DIT(mlp_args, lstm_args)
decoder = decoder.to(device) # use GPU


# Training

#learning_rate = 0.001 #? # 0.01 was not helping (the epoch loss was not decreasing)
#print(decoder.parameters())
optimizer = optim.Adam([
    {"params":decoder.parameters(), "lr":0.0005,},
    {"params":lat_vecs.parameters(), "lr":0.001,}, ] )

print("optimizer",optimizer)
# TODO shuffle the data somehow
# TODO split data into training and test data


mini_batch_size =5000 
num_epochs = 2000 #2001
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    epoch_loss = 0
    decoder.train()
 #   print("lat_vecs",lat_vecs(torch.IntTensor([0])))
    for a, (shape_data, latent_code_index) in enumerate(sdf_loader): # for each shape
        shape_start_time = time.time()
        # loose first dimension (shape dimension)
        data = shape_data.data.reshape(-1, 4)
       # print("data.shape", shape_data.shape)

        num_samples = data.shape[0]
       # print("number of samples", num_samples)

        # format data for the whole shape
        xyz = data[:,0:3] # point coordinates
        xyz.requires_grad = False
        sdf_gt = data[:,3] # ground truth sdf values
        sdf_gt.requires_grad = False
        sdf_gt = torch.clamp(sdf_gt, -0.1, 0.1)
        
        lat_indices = latent_code_index.repeat(num_samples) # repeated latent code index

        shape_loss = 0
        
        num_batches = num_samples // mini_batch_size
        for i in range(num_batches):
            start_batch_time = time.time()
            #print('batches number:', i)
            # for each batch
            start_index = i * mini_batch_size
            end_index = start_index + mini_batch_size
            if(end_index > num_samples):
                end_index = num_samples

            # get batch data
            batch_xyz = xyz[start_index: end_index].to(device) # point coordinates
            batch_sdf_gt = sdf_gt[start_index: end_index].to(device) # ground truth values
            batch_lat_indices = lat_indices[start_index: end_index]
            batch_lat_codes = lat_vecs(batch_lat_indices).to(device) # latent codes
            input = torch.cat([batch_lat_codes, batch_xyz], dim=1)
            
            #print("input before",input)
            #print("batch_sdf_gt",batch_sdf_gt)
            # feed input into network
            sdf_values, warped_points = decoder(input)
            for i in range(len(sdf_values)):
                sdf_values[i] = torch.clamp(sdf_values[i], -0.1, 0.1)
            #print("sdf_values",sdf_values)
            # compute loss
            start_loss_time = time.time()
            rec_loss = reconstruction_loss(sdf_values, batch_sdf_gt)
            
            end_rec_time = time.time()
            
            reg_loss = regularization_loss(batch_xyz, warped_points[-1], batch_lat_codes, epoch) # final_positions
            
            end_loss_time = time.time()
            
         #   print("REC LOSS TIME:",end_rec_time-start_loss_time)
         #   print("REG LOSS TIME:",end_loss_time-end_rec_time)

            #print("REC LOSS:",rec_loss.item(),"REG LOSS:",reg_loss.item())#,"min sdf: ",torch.min(sdf_values[1]))
            #print("REC LOSS:",rec_loss,"REG LOSS:",reg_loss)
            batch_loss = rec_loss + reg_loss # / (end_index - start_index)
            shape_loss += batch_loss.item() 
            # / (end_index - start_index)

            start_optim_time = time.time()
            # flush out previously computed gradient
            optimizer.zero_grad()

            # backpropagation
            batch_loss.backward()
            
#             print("input middle",input)
#             print("-------")
#             for name, param in decoder.named_parameters():
#                 print(name, param.grad)

            # update network weights
            optimizer.step()
            
     #       print("input after",input)
            end_optim_time = time.time()
          #  print("OPTIM TIME:", end_optim_time-start_optim_time)

            
            end_batch_time = time.time()
       #     print("BATCH TIME:",end_batch_time-start_batch_time)
            #print("after batch", i)
            # release memory
            del sdf_values, warped_points, input, batch_xyz, batch_sdf_gt, batch_lat_indices, batch_lat_codes, start_index, end_index#, rec_loss, reg_loss,final_positions
            torch.cuda.empty_cache()
        # update tensorboard
        writer.add_scalar("train_loss", shape_loss, a)
        
        shape_end_time = time.time()
#        if  a%100 == 0
 #           print("Loss for SHAPE (batch)", a, " is:", shape_loss, " computed in:", shape_end_time-shape_start_time, " seconds")
        
        epoch_loss += shape_loss

        # print train loss, accuracy, time
        del shape_loss, data, xyz, sdf_gt, lat_indices
        
    # save weights in weights_file
    # where to save model weights
    weights_file = './weights/weights_%d.pt'
    if epoch % 10 == 0:
        weights_file_1 = weights_file%epoch
        torch.save(decoder.state_dict(),weights_file_1)
    
    torch.save(decoder.state_dict(),"./weights/weight_latest.pt")

    epoch_end_time = time.time()
    print("Epoch", epoch, " ended after :", epoch_end_time-epoch_start_time, "seconds with epoch loss:", epoch_loss)

optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0005
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Epoch 0  ended after : 148.74156737327576 seconds with epoch loss: 12.280261078441981
Epoch 1  ended after : 147.39031457901 seconds with epoch loss: 1.4838129887357354
Epoch 2  ended after : 148.27231550216675 seconds with epoch loss: 1.259433196275495
Epoch 3  ended after : 149.38356828689575 seconds with epoch loss: 1.1477044753846712
Epoch 4  ended after : 150.47797775268555 seconds with epoch loss: 1.1428906040964648
Epoch 5  ended after : 148.46065497398376 seconds with epoch loss: 1.0542296629282646
Epoch 6  ended after : 147.29500269889832 seconds with epoch loss: 1.0261656538932584
Epoch 7  ended after : 147.5765597820282 seconds with epoch loss: 1.0263275767792948
Epoch 8  ended after : 148.5672676563263 seconds with epoch loss: 1.00739322